# ULTRATHINK Model Training - Google Colab

This notebook provides a complete training pipeline for the ULTRATHINK model on Google Colab.

## Features
- ✅ Mixture of Experts (MoE)
- ✅ Dynamic Reasoning Engine (DRE)
- ✅ Constitutional AI
- ✅ RLHF (optional)
- ✅ Multimodal capabilities (optional)
- ✅ MLflow tracking
- ✅ Google Drive integration for checkpoints

## Requirements
- GPU Runtime (T4/V100/A100)
- High RAM option recommended

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Mount Google Drive (for saving checkpoints)
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint directory
!mkdir -p /content/drive/MyDrive/ULTRATHINK_checkpoints

In [ ]:
# Clone repository
!git clone https://github.com/vediyappanm/UltraThinking-LLM-Training.git
%cd UltraThinking-LLM-Training

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

# Install flash attention (if supported)
try:
    !pip install -q flash-attn --no-build-isolation
    print("✅ Flash Attention installed")
except:
    print("⚠️  Flash Attention not available (will use standard attention)")

## 2. Configure MLflow API Key (Optional)

In [ ]:
import os

# Option 1: Local MLflow (default)
os.environ['MLFLOW_TRACKING_URI'] = 'file:./mlruns'

# Option 2: Remote MLflow server (uncomment and configure)
# os.environ['MLFLOW_TRACKING_URI'] = 'https://your-mlflow-server.com'
# os.environ['MLFLOW_TRACKING_USERNAME'] = 'your-username'
# os.environ['MLFLOW_TRACKING_PASSWORD'] = 'your-password'

## 3. Training Configurations

Choose your training configuration based on available GPU resources.

### Configuration 1: Small Model (T4 GPU, 16GB RAM)

In [ ]:
# Quick test training
!python train_advanced.py \
  --config configs/train_small.yaml \
  --override \
    training.batch_size=2 \
    training.gradient_accumulation_steps=16 \
    model.max_seq_length=512 \
    data.train_samples=10000 \
    data.val_samples=1000 \
    training.num_epochs=1 \
    logging.use_mlflow=true \
    output.output_dir=/content/drive/MyDrive/ULTRATHINK_checkpoints/small_model \
  --run-name "colab_small_test"

### Configuration 2: Medium Model (V100/A100 GPU)

In [ ]:
# Medium model with advanced features
!python train_advanced.py \
  --config configs/train_medium.yaml \
  --override \
    training.batch_size=4 \
    training.gradient_accumulation_steps=8 \
    advanced.enable_moe=true \
    advanced.enable_dre=true \
    advanced.enable_constitutional=true \
    data.streaming=false \
    data.train_samples=50000 \
    data.val_samples=5000 \
    training.num_epochs=2 \
    logging.use_mlflow=true \
    output.output_dir=/content/drive/MyDrive/ULTRATHINK_checkpoints/medium_model \
  --run-name "colab_medium_training"

### Configuration 3: Large Model (A100 40/80GB)

In [ ]:
# Full-scale training with all features
!python train_advanced.py \
  --config configs/train_large.yaml \
  --override \
    training.batch_size=8 \
    advanced.enable_moe=true \
    advanced.enable_dre=true \
    advanced.enable_constitutional=true \
    advanced.enable_multimodal=true \
    data.streaming=true \
    logging.use_mlflow=true \
    output.output_dir=/content/drive/MyDrive/ULTRATHINK_checkpoints/large_model \
  --run-name "colab_large_training"

## 4. Custom Training with Command-Line Interface

Alternatively, use the original training script:

In [ ]:
# Custom training with specific parameters
!python train_ultrathink.py \
  --use_mlflow \
  --mlflow_experiment "ULTRATHINK-Colab" \
  --run_name "custom_colab_run" \
  --dataset wikitext \
  --hidden_size 1024 \
  --num_layers 12 \
  --num_heads 16 \
  --num_kv_heads 8 \
  --batch_size 4 \
  --gradient_accumulation_steps 8 \
  --max_seq_length 2048 \
  --learning_rate 1e-4 \
  --num_epochs 2 \
  --enable_moe \
  --enable_dre \
  --enable_constitutional \
  --use_flash_attention \
  --gradient_checkpointing \
  --use_amp \
  --eval_frequency 1 \
  --output_dir /content/drive/MyDrive/ULTRATHINK_checkpoints/custom_model

## 5. Monitor Training

### View Logs

In [ ]:
# View training logs
!tail -n 50 ./outputs/*/training.log

### MLflow UI (Local)

To view MLflow metrics in Colab, use ngrok tunnel:

In [ ]:
# Install ngrok
!pip install -q pyngrok

# Start MLflow UI in background
import subprocess
import time
from pyngrok import ngrok

# Start MLflow server
mlflow_proc = subprocess.Popen(["mlflow", "ui", "--port", "5000"])
time.sleep(3)

# Create ngrok tunnel
public_url = ngrok.connect(5000)
print(f"\n🚀 MLflow UI available at: {public_url}\n")

### TensorBoard (Alternative)

In [ ]:
# Load TensorBoard extension
%load_ext tensorboard
%tensorboard --logdir ./outputs

## 6. Resume Training from Checkpoint

In [ ]:
# Resume from saved checkpoint
!python train_advanced.py \
  --config configs/train_medium.yaml \
  --resume /content/drive/MyDrive/ULTRATHINK_checkpoints/medium_model/checkpoint_epoch_0.pt \
  --run-name "resumed_training"

## 7. Fine-tuning with RLHF

In [ ]:
# Fine-tune with RLHF
!python train_advanced.py \
  --config configs/train_medium.yaml \
  --init-from /content/drive/MyDrive/ULTRATHINK_checkpoints/medium_model/final_model \
  --override \
    advanced.enable_rlhf=true \
    training.learning_rate=1e-5 \
    training.num_epochs=1 \
    rlhf.rlhf_frequency=1 \
    output.output_dir=/content/drive/MyDrive/ULTRATHINK_checkpoints/rlhf_model \
  --run-name "rlhf_finetuning"

## 8. Evaluation

In [ ]:
# Evaluate trained model
import json

# Load evaluation results
with open('./outputs/*/evaluation_results.json', 'r') as f:
    results = json.load(f)

print("\n📊 Evaluation Results:")
print(json.dumps(results, indent=2))

## 9. Export Model for Inference

In [ ]:
# Export model in HuggingFace format
import torch
from src.models.ultrathink import UltraThinkModel

# Load trained model
model_path = "/content/drive/MyDrive/ULTRATHINK_checkpoints/medium_model/final_model"
model = UltraThinkModel.from_pretrained(model_path)

# Save for deployment
export_path = "/content/drive/MyDrive/ULTRATHINK_checkpoints/exported_model"
model.save_pretrained(export_path)

print(f"✅ Model exported to: {export_path}")

## 10. Quick Inference Test

In [ ]:
# Quick inference test
from transformers import AutoTokenizer
import torch

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = UltraThinkModel.from_pretrained(export_path)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Test generation
prompt = "The future of artificial intelligence is"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(
        inputs["input_ids"],
        max_length=100,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.9
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\n🤖 Generated Text:\n{generated_text}")

## 11. Download Results

In [ ]:
# Zip and download results
from google.colab import files
import shutil

# Create archive
shutil.make_archive('training_results', 'zip', './outputs')

# Download
files.download('training_results.zip')
print("✅ Results downloaded!")

## Tips & Troubleshooting

### Out of Memory (OOM) Errors
- Reduce `batch_size`
- Increase `gradient_accumulation_steps`
- Reduce `max_seq_length`
- Enable `gradient_checkpointing`
- Disable advanced features temporarily

### Slow Training
- Enable `use_flash_attention` if supported
- Use larger `batch_size` if memory allows
- Enable `use_amp` for mixed precision
- Reduce `num_workers` if CPU bottleneck

### Connection Issues
- Save checkpoints frequently to Google Drive
- Use `--continuous` flag for long training
- Keep browser tab active to prevent disconnection

## Resources
- 📖 [Documentation](https://github.com/vediyappanm/UltraThinking-LLM-Training)
- 💬 [Issues](https://github.com/vediyappanm/UltraThinking-LLM-Training/issues)
- 📊 [Model Card](MODEL_CARD.md)
